In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!unzip "gdrive/My Drive/Tesi/PD_Detection/dataset.zip" -d .
!mv dataset data

In [ ]:
%cp gdrive/MyDrive/Tesi/PD_Detection/*.py .

In [ ]:
!ls *py

Import libreries

In [ ]:
import config as cfg
import train
import predict
import data_manager
import evaluator
import tensorflow as tf

from sklearn.utils import resample

import logging

tf.get_logger().setLevel(logging.ERROR)

In [ ]:
np.subtract(2,4)

-2

In [ ]:
import numpy as np
from imblearn.over_sampling import RandomOverSampler

data_manager.split_train_test_by_subject()
train_i = data_manager.get_train_inds()

data = data_manager.load_data("rest", train_i, "acc")
labels = data_manager.load_labels(train_i)

print(data.shape)
print(labels.shape)
print()

values, counts = np.unique(labels, return_counts=True)

minority_class = bool(np.argmin(counts))
samples_to_add = abs(counts[0] - counts[1])
minority_inds = np.where(labels == minority_class)

new_samples = resample(data[minority_inds], n_samples = samples_to_add)
new_labels = np.repeat(minority_class, samples_to_add)

print(new_samples.shape)
print(new_labels.shape)
print()

data = np.concatenate((data,new_samples))
labels = np.concatenate((labels,new_labels))

print(data.shape)
print(labels.shape)
print()



Start main loop

In [ ]:
if __name__ == "__main__":
    
    # data_manager.create_fromcsv_files(cfg.CSV_PATH)
    
    n_folds = cfg.FOLDS    
    evaluations = []
    evaluations_ind_mean = []
    evaluations_ind_max = []
    
    
    for fold in range(n_folds):
        
        res_path = "results/test/fold{}/".format(fold)
        
        data_manager.split_train_test_by_subject()
        
        train.run(fold)
        predict.run(fold)
        
        eva = evaluator.evaluate(res_path + "predictions.npy", res_path + "labels.npy")
        eva_ind_mean = evaluator.evaluate(res_path + "predictions_ind_mean.npy", res_path + "labels.npy")
        eva_ind_max = evaluator.evaluate(res_path + "predictions_ind_max.npy", res_path + "labels.npy")
        
        eva.save_evaluation(res_path + "eva.txt")
        eva_ind_mean.save_evaluation(res_path + "eva_ind_mean.txt")
        eva_ind_max.save_evaluation(res_path + "eva_ind_max.txt")
        
        evaluations.append(eva)
        evaluations_ind_mean.append(eva_ind_mean)
        evaluations_ind_max.append(eva_ind_max)
        
    evaluator.average_evaluations(evaluations, file = "results/test/avg_eva.txt")
    evaluator.average_evaluations(evaluations_ind_mean, file = "results/test/avg_eva_ind_mean.txt")
    evaluator.average_evaluations(evaluations_ind_max, file = "results/test/avg_eva_ind_max.txt")

In [ ]:
!rm -r dataset

In [ ]:
!cp -r results gdrive/MyDrive/Tesi/PD_Detection/
!cp -r logs gdrive/MyDrive/Tesi/PD_Detection/
!cp -r trained\ models gdrive/MyDrive/Tesi/PD_Detection/